<a href="https://colab.research.google.com/github/klordo/nlp_homeworks/blob/hw1/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

До этого не работал в колабе. Читал правила красивого написания для ноутбуков. Как я понял, эти правила для проектов-ноутбуков, но отчасти не для колаба. Потому установки зависимостей делал перед каждым инструментом. Могу исправить и переместить вверх.

Пробовал различные инструменты, оставил два варианта:
Первый через pymorphy2 и nltk без удаления стоп слов.
Второй через spacy.
Не решил одну проблему, о которой написал в самом конце.

Если нужны будут исправления, то хотел бы доделать до максимального балла, если это возможно)

In [2]:
!pip install corus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 4.3 MB/s eta 0:00:00


In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2

In [162]:
from corus import load_lenta2

path = 'lenta-ru-news.csv.bz2'
records = load_lenta2(path)

In [163]:
dataset = [next(records).text for i in range(1000)]
dataset[0]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. «Русский инвалид», 16 сентября 1914 года.'

In [164]:
import re

In [165]:
email_pattern = r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+'
url_pattern = r'https?://\S+|www\.\S+|\b\w+\.\w+\b'

if all([re.search(email_pattern, text) is None for text in dataset]):
    print('Email не найдены')
else:
    print('Email найдены')

Email найдены


In [166]:
dataset_clean = [re.sub(email_pattern, '<email>', text) for text in dataset]
dataset_clean = [re.sub(url_pattern, '<url>', text) for text in dataset]

nltk и pymorphy2

In [167]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [168]:
from nltk.tokenize import (
    sent_tokenize,
    word_tokenize,
    TweetTokenizer,
    WordPunctTokenizer,
    WhitespaceTokenizer,
    LegalitySyllableTokenizer,
    SyllableTokenizer,
    MWETokenizer,
)

Данным инструментом решил разделить список документов по спискам предложений, каждый из которых делится на списки слов.

In [169]:
dataset_tokenized = [[word_tokenize(sent, language='russian') for sent in sent_tokenize(text, language='russian')] for text in dataset_clean]
dataset_tokenized[0][:2]

[['Бои',
  'у',
  'Сопоцкина',
  'и',
  'Друскеник',
  'закончились',
  'отступлением',
  'германцев',
  '.'],
 ['Неприятель',
  ',',
  'приблизившись',
  'с',
  'севера',
  'к',
  'Осовцу',
  'начал',
  'артиллерийскую',
  'борьбу',
  'с',
  'крепостью',
  '.']]

С помощью pymorphy2 провел лемматизацию, но не смог удалить стоп слова.

In [ ]:
!pip install pymorphy2

In [31]:
import pymorphy2

In [170]:
morph = pymorphy2.MorphAnalyzer()

In [171]:
dataset_lemmatized = [[[morph.parse(word)[0].normal_form for word in sent] for sent in text] for text in dataset_tokenized]

In [172]:
dataset_lemmatized[0][0]

['бой',
 'у',
 'сопоцкина',
 'и',
 'друскеник',
 'закончиться',
 'отступление',
 'германец',
 '.']

Вариант с помощью spacy для токенизации, лемматизации и удаления стоп слов.

In [67]:
import spacy

In [ ]:
!python3 -m spacy download ru_core_news_sm

In [69]:
nlp = spacy.load("ru_core_news_sm")

In [173]:
dataset_doc = [[token.lemma_ for token in nlp(doc) if not token.is_stop] for doc in dataset_clean]

In [174]:
dataset_doc[0][:10]

['бой',
 'сопоцкина',
 'друскеник',
 'закончиться',
 'отступление',
 'германец',
 '.',
 'неприятель',
 ',',
 'приблизиться']

Далее - очистка чисел. Если для модели они могут понадобится, то можно провести более детальную очистку.

In [175]:
for doc in dataset_doc[:2]:
  for token in doc:
    if re.search(r'\d', token):
      print(token)

14
16
1914
2-й
1914
16
1914


In [176]:
dataset_doc = [[word for word in doc if not re.search(r'\d', word)] for doc in dataset_doc]

Не решенная проблема - многие слова не разделены пробелами.

In [177]:
for doc in dataset_doc[:10]:
  for token in doc:
    if len(token) > 20:
      print(token)

воротамкенсингтонского
долларов).средичленов
сдержаннойпоминальной
оглашеныокончательные
французскиеследователи


In [141]:
!pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=4b5b463326ba896e7ac4393e8e3d199f8a9725e4c5336b5442d2c4017ea2c316
  Stored in directory: /root/.cache/pip/wheels/aa/44/3a/f2a5c1859b8b541ded969b4cd12d0a58897f12408f4f51e084
Successfully built wordninja


In [142]:
import wordninja

In [156]:
text = 'этоттекстнеразделенпробелами'
words = wordninja.split(text)
words

[]

Необходимо перевести модель на русскую, но я пока не нашел способ как ее скачать в коллаб...